In [2]:
# prompt: mount drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import time
import os
import pandas as pd
import json
from PIL import Image

In [3]:

save_dir = '/content/drive/MyDrive/image-classifications/classification_results'
os.makedirs(save_dir, exist_ok=True)
print(f"All results will be saved in: {save_dir}")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

with open(f'{save_dir}/device_info.txt', 'w') as f:
    f.write(str(device))

config = {
    'batch_size': 64,
    'learning_rate': 0.001,
    'num_epochs': 10,
    'model_architecture': 'FC-512-256-10',
    'optimizer': 'Adam',
    'loss_function': 'CrossEntropyLoss'
}

# Save config
with open(f'{save_dir}/config.json', 'w') as f:
    json.dump(config, f, indent=4)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

All results will be saved in: /content/drive/MyDrive/image-classifications/classification_results
Using device: cuda


In [4]:
# MNIST dataset
train_dataset = torchvision.datasets.MNIST(
    root='./data',
    train=True,
    transform=transform,
    download=True
)

test_dataset = torchvision.datasets.MNIST(
    root='./data',
    train=False,
    transform=transform,
    download=True
)

# data loaders
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=config['batch_size'],
    shuffle=True
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=config['batch_size'],
    shuffle=False
)

In [5]:
def save_sample_images(loader, save_path, num_images=8):
    images, labels = next(iter(loader))
    img_grid = torchvision.utils.make_grid(images[:num_images])
    npimg = img_grid.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)).squeeze(), cmap='gray')
    plt.savefig(save_path)
    plt.close()

    os.makedirs(f'{save_dir}/sample_images', exist_ok=True)
    for i in range(num_images):
        img = images[i].squeeze().numpy()
        img = (img * 255).astype(np.uint8)
        im = Image.fromarray(img)
        im.save(f'{save_dir}/sample_images/train_sample_{i}_label_{labels[i]}.png')

save_sample_images(train_loader, f'{save_dir}/training_samples.png')

In [6]:
class MNISTClassifier(nn.Module):
    def __init__(self):
        super(MNISTClassifier, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28*28, 512)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

In [7]:
model = MNISTClassifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])

In [8]:
with open(f'{save_dir}/model_architecture.txt', 'w') as f:
    f.write(str(model))

In [9]:
def train(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    history = {
        'epoch': [],
        'train_loss': [],
        'train_acc': [],
        'epoch_time': []
    }

    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        start_time = time.time()

        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(train_loader)
        epoch_acc = 100 * correct / total
        epoch_time = time.time() - start_time

        history['epoch'].append(epoch + 1)
        history['train_loss'].append(epoch_loss)
        history['train_acc'].append(epoch_acc)
        history['epoch_time'].append(epoch_time)

        print(f'Epoch [{epoch+1}/{num_epochs}], '
              f'Loss: {epoch_loss:.4f}, '
              f'Accuracy: {epoch_acc:.2f}%, '
              f'Time: {epoch_time:.2f}s')

    pd.DataFrame(history).to_csv(f'{save_dir}/training_history.csv', index=False)

    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history['epoch'], history['train_loss'], label='Training Loss')
    plt.title('Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history['epoch'], history['train_acc'], label='Training Accuracy')
    plt.title('Training Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.legend()
    plt.savefig(f'{save_dir}/training_curves.png')
    plt.close()

    return history

In [10]:
print("Starting training...")
train_history = train(model, train_loader, criterion, optimizer, config['num_epochs'])

Starting training...
Epoch [1/10], Loss: 0.2051, Accuracy: 93.66%, Time: 14.66s
Epoch [2/10], Loss: 0.0848, Accuracy: 97.39%, Time: 12.96s
Epoch [3/10], Loss: 0.0610, Accuracy: 98.07%, Time: 12.88s
Epoch [4/10], Loss: 0.0476, Accuracy: 98.44%, Time: 12.95s
Epoch [5/10], Loss: 0.0402, Accuracy: 98.69%, Time: 12.99s
Epoch [6/10], Loss: 0.0301, Accuracy: 99.00%, Time: 19.10s
Epoch [7/10], Loss: 0.0289, Accuracy: 99.07%, Time: 16.84s
Epoch [8/10], Loss: 0.0250, Accuracy: 99.23%, Time: 13.02s
Epoch [9/10], Loss: 0.0219, Accuracy: 99.28%, Time: 12.99s
Epoch [10/10], Loss: 0.0200, Accuracy: 99.32%, Time: 13.17s


In [13]:
def evaluate(model, test_loader):
    model.eval()
    all_labels = []
    all_predictions = []
    all_probabilities = []
    test_images = []

    with torch.no_grad():
        correct = 0
        total = 0
        test_loss = 0.0

        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            probabilities = torch.nn.functional.softmax(outputs, dim=1)
            loss = criterion(outputs, labels)
            test_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())
            all_probabilities.extend(probabilities.cpu().numpy())
            test_images.extend(images.cpu().numpy())

        test_loss /= len(test_loader)
        test_acc = 100 * correct / total

        print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.2f}%')

        test_metrics = {
            'test_loss': test_loss,
            'test_accuracy': test_acc,
            'correct_predictions': correct,
            'total_samples': total
        }
        with open(f'{save_dir}/test_metrics.json', 'w') as f:
            json.dump(test_metrics, f, indent=4)

        results_df = pd.DataFrame({
            'true_label': all_labels,
            'predicted_label': all_predictions,
            'probabilities': list(all_probabilities)
        })
        results_df.to_csv(f'{save_dir}/test_predictions.csv', index=False)

        os.makedirs(f'{save_dir}/test_images', exist_ok=True)
        for i in range(min(20, len(test_images))):
            img = test_images[i].squeeze()
            img = (img * 255).astype(np.uint8)
            im = Image.fromarray(img)

            true_label = all_labels[i]
            pred_label = all_predictions[i]
            prob = max(all_probabilities[i]) * 100

            im.save(f'{save_dir}/test_images/test_{i}_true_{true_label}_pred_{pred_label}_prob_{prob:.1f}.png')

        return all_labels, all_predictions, all_probabilities, test_loss, test_acc

In [14]:
print("Evaluating on test set...")
true_labels, pred_labels, probabilities, test_loss, test_acc = evaluate(model, test_loader)

# classification report
class_report = classification_report(true_labels, pred_labels, target_names=[str(i) for i in range(10)], output_dict=True)
print("\nClassification Report:")
print(classification_report(true_labels, pred_labels, target_names=[str(i) for i in range(10)]))

with open(f'{save_dir}/classification_report.json', 'w') as f:
    json.dump(class_report, f, indent=4)

Evaluating on test set...
Test Loss: 0.1003, Test Accuracy: 97.82%

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       980
           1       0.99      0.98      0.99      1135
           2       0.99      0.98      0.98      1032
           3       0.97      0.98      0.97      1010
           4       0.98      0.97      0.98       982
           5       0.98      0.99      0.98       892
           6       0.99      0.98      0.98       958
           7       0.95      0.98      0.97      1028
           8       0.96      0.98      0.97       974
           9       0.98      0.95      0.96      1009

    accuracy                           0.98     10000
   macro avg       0.98      0.98      0.98     10000
weighted avg       0.98      0.98      0.98     10000



In [15]:
conf_mat = confusion_matrix(true_labels, pred_labels)
plt.figure(figsize=(10, 8))
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues',
            xticklabels=[str(i) for i in range(10)],
            yticklabels=[str(i) for i in range(10)])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.savefig(f'{save_dir}/confusion_matrix.png')
plt.close()

pd.DataFrame(conf_mat).to_csv(f'{save_dir}/confusion_matrix.csv')

In [16]:
def save_test_predictions(test_loader, model, num_images=16):
    model.eval()
    images, labels = next(iter(test_loader))
    images = images.to(device)
    labels = labels.to(device)

    with torch.no_grad():
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)

    images = images.cpu()
    labels = labels.cpu()
    predicted = predicted.cpu()
    probabilities = probabilities.cpu()

    plt.figure(figsize=(12, 12))
    for i in range(num_images):
        plt.subplot(4, 4, i+1)
        plt.imshow(images[i].squeeze(), cmap='gray')
        prob_percent = probabilities[i][predicted[i]] * 100
        plt.title(f'True: {labels[i]}\nPred: {predicted[i]} ({prob_percent:.1f}%)')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig(f'{save_dir}/test_predictions_visualization.png')
    plt.close()

save_test_predictions(test_loader, model)

In [17]:
torch.save(model.state_dict(), f'{save_dir}/classification_model_weights.pth')

torch.save(model, f'{save_dir}/classification_model_full.pth')